In [23]:
! pip install -U tensorflow keras

     |████████████████████████████████| 86.3MB 733bytes/s  0:00:01    |▋                               | 1.6MB 1.9MB/s eta 0:00:45     |██▊                             | 7.3MB 1.5MB/s eta 0:00:52     |███▏                            | 8.5MB 796kB/s eta 0:01:38     |████▋                           | 12.4MB 1.1MB/s eta 0:01:07     |█████████▌                      | 25.6MB 1.2MB/s eta 0:00:49     |█████████▌                      | 25.7MB 1.2MB/s eta 0:00:49     |█████████▋                      | 25.9MB 785kB/s eta 0:01:17     |█████████▉                      | 26.6MB 236kB/s eta 0:04:13     |██████████                      | 26.7MB 236kB/s eta 0:04:12     |██████████▊                     | 29.0MB 4.3MB/s eta 0:00:14     |███████████████                 | 40.4MB 102kB/s eta 0:07:27     |█████████████████▌              | 47.2MB 89kB/s eta 0:07:18     |████████████████████▏           | 54.4MB 127kB/s eta 0:04:11     |██████████████████████          | 59.3MB 69kB/s eta 0:06:32     |██████████

  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7637 sha256=41af8f4f2dc1f4089fea060b4a4ea9d7a3c68e19c850d205085840b644e8e6bb
  Stored in directory: /home/nbuser/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp36-none-any.whl size=63907 sha256=5c55be79c467b10b06780547ce261635dbed47e7a90b4120d9c550e0cd944ade
  Stored in directory: /home/nbuser/.cache/pip/wheels/2c/b1/94/43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
Successfully built gast opt-einsum
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2
  Found existing installation: tensorflow 2.0.0b1
    Uninstalling tensorflow-2.0.0b1:
      Successfully uninstalled tensorflow-2.0.0b1


In [24]:
! pip install -U backtrader

Requirement already up-to-date: backtrader in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (1.9.74.123)


In [25]:
import numpy
import pandas
import tensorflow
import keras
import sys


def Im_Lstm_Model_Initialization_Funcion(FeatureData_PeriodRange_Start, FeatureData_PeriodRange_End ):

    Im_cute_model = keras.models.Sequential()
        
    Im_cute_model.add(keras.layers.recurrent.LSTM(256, input_shape=(None, 5*( FeatureData_PeriodRange_Start - FeatureData_PeriodRange_End )), return_sequences=True))
    Im_cute_model.add(keras.layers.core.Dropout(0.1))

    Im_cute_model.add(keras.layers.recurrent.LSTM(512, return_sequences=True))
    Im_cute_model.add(keras.layers.core.Dropout(0.2))

    Im_cute_model.add(keras.layers.recurrent.SimpleRNN(256, return_sequences=True))
    Im_cute_model.add(keras.layers.core.Dropout(0.2))

    Im_cute_model.add(keras.layers.core.Dense(units = 512, activation = "relu"))
    Im_cute_model.add(keras.layers.core.Dropout(0.2))

    Im_cute_model.add(keras.layers.recurrent.LSTM(256, return_sequences=True))
    Im_cute_model.add(keras.layers.core.Dropout(0.2))

    Im_cute_model.add(keras.layers.recurrent.LSTM(128, return_sequences=True))
    Im_cute_model.add(keras.layers.core.Dropout(0.1))

    Im_cute_model.add(keras.layers.core.Dense(units = 1, activation = "linear"))

    Im_AdamAMSgrad_Optimizer = keras.optimizers.Adam(amsgrad=True)

    Im_cute_model.compile(loss = "mse", optimizer = Im_AdamAMSgrad_Optimizer )

    return Im_cute_model

Im_cute_model = Im_Lstm_Model_Initialization_Funcion(-10, -60)


def Im_prediction_function(Im_train_input):

    Im_cute_prediction = Im_cute_model.predict(Im_train_input)

    return Im_cute_prediction

    
def Im_training_function(Im_train_input_set, Im_train_label_set, Im_train_epochs): 

    Im_cute_model.fit(
        Im_train_input_set,
        Im_train_label_set,
#        batch_size = 1,
        epochs = Im_train_epochs,
        verbose = 2,
#        validation_split = 0.05
    )
    return Im_cute_model.evaluate(x =  Im_train_input_set, y =  Im_train_label_set)


def Im_performance_testing_function(Im_train_input_set, Im_train_label_set): 

    return Im_cute_model.evaluate(x =  Im_train_input_set, y =  Im_train_label_set)

In [26]:
#Building Stategy

from datetime import datetime
import backtrader

def Function_For_Build_SupervisedLSTM_StrategyTradingAndTrainingObject(FeatureData_PeriodRange_Start, FeatureData_PeriodRange_End):

    class LSTM_StrategyTradingAndTrainingObject(backtrader.Strategy):
        def __init__(self):

            self.data_open = self.datas[0].open
            self.data_high = self.datas[0].high
            self.data_low = self.datas[0].low
            self.data_close = self.datas[0].close
            self.data_volume = self.datas[0].volume
               
        def next(self):


            def Function_Make_FeatureDataSet(From_Period, To_Period):

                Im_Feature_DataSet = []

                for TimeSequence in range( From_Period, To_Period, -1) :

                        Im_Feature_DataSet.append(self.data_open[TimeSequence])
                        Im_Feature_DataSet.append(self.data_high[TimeSequence])
                        Im_Feature_DataSet.append(self.data_low[TimeSequence])
                        Im_Feature_DataSet.append(self.data_close[TimeSequence])
                        Im_Feature_DataSet.append(self.data_volume[TimeSequence])

                return Im_Feature_DataSet

            Im_Current_Feature_DataSet = numpy.reshape(Function_Make_FeatureDataSet(0, FeatureData_PeriodRange_End - FeatureData_PeriodRange_Start), (1,1, 5*( FeatureData_PeriodRange_Start - FeatureData_PeriodRange_End) ))

            if Im_prediction_function(Im_Current_Feature_DataSet) >= (1.03 * self.data_close[0]) :
                self.buy()
            elif Im_prediction_function(Im_Current_Feature_DataSet) <= (0.97 * self.data_close[0]):
                self.sell()


            def Function_TrainingModel(Use_Data_FromPeriod, Use_Data_ToPeriod):

                print("========================== Start Training ==========================")
                Im_Feature_DataSet = numpy.reshape(Function_Make_FeatureDataSet(Use_Data_FromPeriod,Use_Data_ToPeriod), (1,1, 5*( Use_Data_FromPeriod - Use_Data_ToPeriod) ))
                print("Current Price prediction : ")
                print(Im_prediction_function(Im_Feature_DataSet))
                print("Current Price Reality : ")
                print(self.data_close[0])    
                print("Using Current Price Training : ")
                Im_Label = numpy.reshape(self.data_close[0], (1,1,1))
                print(Im_training_function(Im_Feature_DataSet, Im_Label, 1))
                print("========================== End Training ==========================")
                
            Function_TrainingModel(FeatureData_PeriodRange_Start, FeatureData_PeriodRange_End)


    return LSTM_StrategyTradingAndTrainingObject


In [27]:

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import warnings
warnings.filterwarnings('ignore')

class Trading_Picture_Generator:


    # General
    Trading_Object_Name = ""
    Trading_Level_Multiplier = 1000
    
    Trading_DateTime_Start = datetime(2018,1,1)
    Trading_DateTime_End = datetime(2019,9,22)

    I_Am_Trading_Cerebro_Object = backtrader.Cerebro()


    # Parameter for LSTM
    FeatureData_PeriodRange_Start = -10
    FeatureData_PeriodRange_End = -60

    CurrentModel_TrainingTimes = 0



    def __init__(
        self, 
        Trading_Object_Name_Input, 
        Trading_Level_Multiplier_Input, 
        Trading_DateTime_Start_Input, 
        Trading_DateTime_End_Input):

        self.Trading_Object_Name = Trading_Object_Name_Input
        self.Trading_Level_Multiplier = Trading_Level_Multiplier_Input

        self.Trading_DateTime_Start = Trading_DateTime_Start_Input
        self.Trading_DateTime_End = Trading_DateTime_End_Input



####################################################################################
    # Functions For LSTM 
####################################################################################
    def Function_RunTradingAndTraining_Lstm(self, FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input) : 

        Cerebro_Object = backtrader.Cerebro()
        Cerebro_Object.broker.setcommission(mult=self.Trading_Level_Multiplier)
        LSTM_StrategyObject = Function_For_Build_SupervisedLSTM_StrategyTradingAndTrainingObject(FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input)
        Cerebro_Object.addstrategy(LSTM_StrategyObject)

        Im_Data = backtrader.feeds.YahooFinanceData(
            dataname = self.Trading_Object_Name , 
            fromdate=self.Trading_DateTime_Start, 
            todate=self.Trading_DateTime_End)    

        Cerebro_Object.adddata(Im_Data)
        Cerebro_Object.broker.setcash(10000.0)
        Cerebro_Object.run()

        self.FeatureData_PeriodRange_Start = FeatureData_PeriodRange_Start_Input
        self.FeatureData_PeriodRange_End = FeatureData_PeriodRange_End_Input
        self.Trading_Cerebro_Object = Cerebro_Object


    def Function_RunTrading_Lstm(self, FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input) : 

        Cerebro_Object = backtrader.Cerebro()
        Cerebro_Object.broker.setcommission(mult=self.Trading_Level_Multiplier)
        LSTM_StrategyObject = Function_For_Build_SupervisedLSTM_StrategyTradingObject(FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input)
        Cerebro_Object.addstrategy(LSTM_StrategyObject)

        Im_Data = backtrader.feeds.YahooFinanceData(
            dataname = self.Trading_Object_Name , 
            fromdate=self.Trading_DateTime_Start, 
            todate=self.Trading_DateTime_End)    

        Cerebro_Object.adddata(Im_Data)
        Cerebro_Object.broker.setcash(10000.0)
        Cerebro_Object.run()

        self.FeatureData_PeriodRange_Start = FeatureData_PeriodRange_Start_Input
        self.FeatureData_PeriodRange_End = FeatureData_PeriodRange_End_Input
        self.Trading_Cerebro_Object = Cerebro_Object


    def Function_RunTraining_Lstm(self, FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input) : 

        Cerebro_Object_ForTraining = backtrader.Cerebro()
        LSTM_StrategyObject = Function_For_Build_SupervisedLSTM_StrategyTrainingObject(FeatureData_PeriodRange_Start_Input, FeatureData_PeriodRange_End_Input)
        Cerebro_Object_ForTraining.addstrategy(LSTM_StrategyObject)

        Im_Data = backtrader.feeds.YahooFinanceData(
            dataname = self.Trading_Object_Name , 
            fromdate=self.Trading_DateTime_Start, 
            todate=self.Trading_DateTime_End)    

        Cerebro_Object_ForTraining.adddata(Im_Data)
        Cerebro_Object_ForTraining.run()

        self.FeatureData_PeriodRange_Start = FeatureData_PeriodRange_Start_Input
        self.FeatureData_PeriodRange_End = FeatureData_PeriodRange_End_Input



    def Function_Batch_LSTM(self, Trading_Object_Name_List, Trading_Level_Multiplier_List, FeatureData_FromPeriod, FeatureData_ToPeriod, Total_TrainTimes):

        self.FeatureData_PeriodRange_Start = FeatureData_FromPeriod
        self.FeatureData_PeriodRange_End = FeatureData_ToPeriod
        Im_cute_model = Im_Lstm_Model_Initialization_Funcion(self.FeatureData_PeriodRange_Start, self.FeatureData_PeriodRange_End)

        for Specific_Trading_Object_Counter in range(len(Trading_Object_Name_List)):

            self.Trading_Object_Name = Trading_Object_Name_List[Specific_Trading_Object_Counter]
            self.Trading_Level_Multiplier = Trading_Level_Multiplier_List[Specific_Trading_Object_Counter]
            
            try:

                for Time_Of_Training in range(0, Total_TrainTimes):

                    try:

                        self.CurrentModel_TrainingTimes = Time_Of_Training + 1
                        #self.Function_RunTrading_Lstm(FeatureData_FromPeriod, FeatureData_ToPeriod)
                        #self.Function_RunTraining_Lstm(FeatureData_FromPeriod, FeatureData_ToPeriod)
                        self.Function_RunTradingAndTraining_Lstm(FeatureData_FromPeriod, FeatureData_ToPeriod)
                        self.Function_Plotting_Lstm()


                    except:

                        print(sys.exc_info())
                        print("Saving the training Model for {} times of training ".format(Time_Of_Training))
                        Im_cute_model.save("Supervised_LSTM_BackTrader__Object[{}]_FeatureRange[{}~{}]_TrainTimes[{}].h5".format(
                            self.Trading_Object_Name,
                            self.FeatureData_PeriodRange_Start,
                            self.FeatureData_PeriodRange_End,
                            self.CurrentModel_TrainingTimes))
                        print("Model safely saved for {} times of training ".format(Time_Of_Training))


                Im_cute_model.save("Supervised_LSTM_BackTrader__Object[{}]_FeatureRange[{}~{}]_TrainTimes[{}].h5".format(
                    self.Trading_Object_Name,
                    self.FeatureData_PeriodRange_Start,
                    self.FeatureData_PeriodRange_End,
                    self.CurrentModel_TrainingTimes))
                print("Model safely saved for {} times of training ".format(Total_TrainTimes))

            except:
                print(sys.exc_info())


    def Function_Plotting_Lstm(self) : 

    #    cerebro.plot()
        figure = matplotlib.pyplot.figure(figsize=(24.0, 15.0))
        figure = self.Trading_Cerebro_Object.plot()[0][0]
        figure.savefig('Trading_Plot/Supervised_Learning/TradingLSTM_Object[{}]_Position[{}%]_Multiplier[{}]_TimeTraining[{}]FeatureRange[{}~{}].jpg'.format(
            self.Trading_Object_Name,
            self.Trading_Cerebro_Object.broker.getvalue()/100, 
            self.Trading_Level_Multiplier, 
            self.CurrentModel_TrainingTimes, 
            self.FeatureData_PeriodRange_Start, 
            self.FeatureData_PeriodRange_End))
        matplotlib.pyplot.close(figure)
        matplotlib.pyplot.close("all")



In [28]:
# Build Instance and draw single plot
%matplotlib inline

In [29]:
# Supervised Lstm 
I_Am_Trading_Instance = Trading_Picture_Generator("TSM", 1000, datetime(2012,1,1), datetime(2019,11,16))
I_Am_Trading_Instance.Function_RunTrading_Lstm(-10, -60)
I_Am_Trading_Instance.Function_Plotting_Lstm()

ValueError: Layer sequential_10 was called with an input that isn't a symbolic tensor. Received type: <class 'backtrader.feeds.yahoo.YahooFinanceData'>. Full input: [<backtrader.feeds.yahoo.YahooFinanceData object at 0x7f373a8fe518>]. All inputs to the layer should be tensors.

In [ ]:
## Taiwan Market ##
#TSMC  #MSCI  #T50
I_Am_Trading_Instance.Function_Batch_LSTM(
    Trading_Object_Name_List = ["TSM","EWT", "0050.TW"], 
    Trading_Level_Multiplier_List = [1000,500,1000],
    FeatureData_FromPeriod = -10,
    FeatureData_ToPeriod = -60,
    Total_TrainTimes = 30)

========================== Start Training ==========================
Current Price prediction : 
[[[0.0013852]]]
Current Price Reality : 
38.05
Using Current Price Training : 
Epoch 1/1
 - 16s - loss: 1448.0652
1/1 [==============================] - 2s 2s/step
1446.611328125
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[0.01217661]]]
Current Price Reality : 
38.69
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 1496.1802
1/1 [==============================] - 0s 12ms/step
1494.597900390625
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[0.029285]]]
Current Price Reality : 
38.49
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 1478.5078
1/1 [==============================] - 0s 11ms/step
1476.8587646484375
========================== En

858.0192260742188
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[10.548459]]]
Current Price Reality : 
40.44
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 907.4871
1/1 [==============================] - 0s 30ms/step
873.81982421875
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[10.883685]]]
Current Price Reality : 
38.85
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 782.2629
1/1 [==============================] - 0s 27ms/step
765.5591430664062
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[11.181735]]]
Current Price Reality : 
37.44
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 694.9124
1/1 [===========

 - 0s - loss: 716.2862
1/1 [==============================] - 0s 40ms/step
740.8521728515625
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[14.731401]]]
Current Price Reality : 
41.74
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 760.9158
1/1 [==============================] - 0s 40ms/step
723.0217895507812
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[14.850939]]]
Current Price Reality : 
41.89
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 746.1909
1/1 [==============================] - 0s 38ms/step
724.7059936523438
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[14.969634]]]
Current Price Reality : 
42.03
Usin

[[[17.259176]]]
Current Price Reality : 
39.77
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 472.0506
1/1 [==============================] - 0s 51ms/step
502.20733642578125
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[17.360018]]]
Current Price Reality : 
39.25
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 472.3204
1/1 [==============================] - 0s 19ms/step
474.8035583496094
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[17.460012]]]
Current Price Reality : 
38.89
Using Current Price Training : 
Epoch 1/1
 - 1s - loss: 432.5732
1/1 [==============================] - 0s 63ms/step
455.00445556640625
========================== End Training ==========================
========================== Start Training ===============

 - 0s - loss: 271.0406
1/1 [==============================] - 0s 31ms/step
262.358154296875
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[19.442526]]]
Current Price Reality : 
36.62
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 285.8592
1/1 [==============================] - 0s 30ms/step
292.3558654785156
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[19.521584]]]
Current Price Reality : 
36.77
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 285.3816
1/1 [==============================] - 0s 41ms/step
294.7929992675781
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[19.600464]]]
Current Price Reality : 
36.78
Using

 - 0s - loss: 204.3037
1/1 [==============================] - 0s 22ms/step
199.01133728027344
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[21.252863]]]
Current Price Reality : 
35.11
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 168.3737
1/1 [==============================] - 0s 41ms/step
190.08160400390625
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[21.322992]]]
Current Price Reality : 
35.06
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 172.3044
1/1 [==============================] - 0s 39ms/step
186.79188537597656
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[21.392817]]]
Current Price Reality : 
34.15
U

 - 0s - loss: 276.8880
1/1 [==============================] - 0s 22ms/step
287.31292724609375
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[22.90969]]]
Current Price Reality : 
39.51
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 281.6078
1/1 [==============================] - 0s 19ms/step
273.0789489746094
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[22.984898]]]
Current Price Reality : 
39.63
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 264.3757
1/1 [==============================] - 0s 19ms/step
274.5509948730469
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[23.060421]]]
Current Price Reality : 
40.11
Usin

 - 0s - loss: 282.6396
1/1 [==============================] - 0s 26ms/step
290.0330810546875
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[24.759644]]]
Current Price Reality : 
41.93
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 273.5424
1/1 [==============================] - 0s 25ms/step
292.12091064453125
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[24.838455]]]
Current Price Reality : 
42.42
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 339.7452
1/1 [==============================] - 0s 22ms/step
306.3000793457031
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[24.918568]]]
Current Price Reality : 
42.52
Usi

 - 0s - loss: 170.4364
1/1 [==============================] - 0s 40ms/step
197.64389038085938
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[26.741411]]]
Current Price Reality : 
40.03
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 136.1996
1/1 [==============================] - 0s 20ms/step
174.47857666015625
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[26.820963]]]
Current Price Reality : 
39.39
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 147.7202
1/1 [==============================] - 0s 42ms/step
156.02734375
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[26.898907]]]
Current Price Reality : 
39.33
Using C

 - 0s - loss: 88.4892
1/1 [==============================] - 0s 12ms/step
89.03620147705078
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[28.2441]]]
Current Price Reality : 
37.22
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 89.0542
1/1 [==============================] - 0s 30ms/step
79.64376068115234
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[28.295666]]]
Current Price Reality : 
36.61
Using Current Price Training : 
Epoch 1/1
 - 0s - loss: 78.6816
1/1 [==============================] - 0s 16ms/step
68.27947998046875
========================== End Training ==========================
========================== Start Training ==========================
Current Price prediction : 
[[[28.34686]]]
Current Price Reality : 
35.64
Using Curr

In [ ]:
## US Index ##
#SP500  #DOW  #NASDAQ
I_Am_Trading_Instance.Function_Batch_LSTM(["^GSPC", "^DJI","^IXIC"], [500,1000,500], -10, -60, 30)

In [ ]:
## Exchange ##
#TWD/USD  #TWD/JPY # 10YearBond #EUR/USD #USD/RMB #BTC-USD
I_Am_Trading_Instance.Function_Batch_LSTM(["TWD=X", "TWDJPY=X", "^TNX", "EURUSD=X", "CNH=F","BTC-USD"], [5000,10000, 1000, 100, 1000,10],  -10, -60, 30)


In [ ]:
## CFTC ##
#GOLD #Silver #Crude Oil
I_Am_Trading_Instance.Function_Batch_LSTM(["GC=F", "SI=F","OIL"], [300, 1000,20000], -10, -60, 30)
